## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import numpy as np
import nltk
import re
import warnings
warnings.filterwarnings("ignore")
from nltk.stem import *
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
train = open('news_train.txt', encoding='utf-8')
test = open('news_test.txt', encoding='utf-8')
stemmer = SnowballStemmer("russian")
def tokenize(file):
    rubrics = []
    texts = []
    tokens = []
    for row in file:
        row = row.strip()
        row = row.lower()
        row = row.split('\t')
        rubric = re.sub(r"[ ]+", ' ', re.sub(r"[^a-zа-я\ ]+", '', row[0])).strip()
        text = re.sub(r"[ ]+", ' ', re.sub(r"[^a-zа-я\ ]+", '', row[1] + row[2])).strip()  
        rubrics.append(rubric)
        texts.append(text)
        tokens.append([stemmer.stem(token) for token in nltk.word_tokenize(text)])
    return rubrics, texts, tokens

In [3]:
test_rubrics, test_texts, test_tokens = tokenize(test)
train_rubrics, train_texts, train_tokens = tokenize(train)

In [4]:
model = Word2Vec(train_tokens)
model.wv.most_similar('ведьмак')

[('effect', 0.8633834719657898),
 ('крофт', 0.8615210056304932),
 ('мультиплеер', 0.8530568480491638),
 ('rpg', 0.851203203201294),
 ('skyrim', 0.8508604764938354),
 ('zombies', 0.8499340415000916),
 ('fable', 0.8497073650360107),
 ('mass', 0.8494325876235962),
 ('экшн', 0.8466614484786987),
 ('diablo', 0.8462177515029907)]

In [5]:
vectorizer = TfidfVectorizer(max_features = 10000)
train_vectors = vectorizer.fit_transform(train_texts).toarray()
test_vectors = vectorizer.transform(test_texts).toarray()

In [6]:
model = MultinomialNB().fit(train_vectors, np.array(train_rubrics))
accuracy_score(model.predict(test_vectors), test_rubrics)

0.8356666666666667

In [7]:
model = LogisticRegression().fit(train_vectors, np.array(train_rubrics))
accuracy_score(model.predict(test_vectors), test_rubrics)

0.8643333333333333